In [1]:
import asyncio
# from functools import wraps

def go_loop(t=0.001):
    def wrapper(func):
        # @wraps(func)
        async def wrapped(*args, **kwargs):
            while 1:
                await asyncio.sleep(t)
                ret=await func(*args, **kwargs)
                if ret:break
            return
        return wrapped
    return wrapper

def to_pop(li):
    def poper(ret_len=5):
        p=li[:ret_len]
        li.pop(0)
        return p
    return poper

def add_num(func):
    num=-1
    def go_num():
        nonlocal num
        num+=1
        # r=num,*func()
        # incompatible
        r=[num]
        r.extend(func())
        return r
    return go_num

class Buff(list):
    def __init__(self, iterable=(),maxlen=128):
        self.maxlen=maxlen
        assert len(iterable)<=maxlen
        super().__init__(iterable)
    def extend(self):
        raise NotImplementedError
    
    def append(self, item):
        if len(self)>=self.maxlen:
            self.pop(0)
        super().append(item)

In [2]:
class Call:
    last_call=0
    
    @go_loop(0.1)
    async def dummy(self):
        if self.last_call>0:self.last_call-=1
C=Call()

In [3]:
f=asyncio.create_task(C.dummy())

In [4]:
def foo():
    return getattr(C,"last_call")

class Watch:    
    def __init__(self,watch_func):
        self.last=False
        self.watch_func=watch_func
    
    @go_loop(0.1)
    async def foo(self):
        if self.watch_func()<=0:
            if self.last:
                print("do_stuff")
                self.last=False
        else:self.last=True
        

In [11]:
C.last_call=10
C.last_call

10

do_stuff


In [6]:
C.last_call

10

In [7]:
w=Watch(foo)

In [8]:
ff=asyncio.create_task(w.foo())

In [9]:
ff

<Task pending coro=<go_loop.<locals>.wrapper.<locals>.wrapped() running at /tmp/ipykernel_355/4206279789.py:7>>

In [10]:
ff.exception

<function Task.exception()>